In [1]:
import numpy as np
import heapq

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
torch.__version__

'1.1.0'

使用pytorch定义网络，2层卷积层、2层池化层、3层全连接层，最终未连接softmax(因为pytorch的交叉熵损失函数内部包含有softmax)，训练时使用单张图像

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #定义两个卷积层、两个池化层、三个全连接层
        #CIFAR-10数据集图像大小为(32,32,3)
        self.conv1 = nn.Conv2d(3, 6, 5)
        #conv1输入3个通道(RGB)，有6个卷积核，卷积核大小为5*5
        #输出6个通道(每个卷积核一个)，28*28的数据。计算公式：(img_size - kernel_size + 2 * padding)/stride + 1。结果为(32-5)/1 + 1= 28 
        #输出(6,28,28)
        self.pool = nn.MaxPool2d(2, 2)#2*2最大池化
        #第一次池化，输出(6,14,14)。28 / 2 = 14
        #第二次池化，输出(16,5,5)。10 / 2 = 5
        self.conv2 = nn.Conv2d(6, 16, 5)
        #conv1输入6个通道，有16个卷积核，卷积核大小为5*5
        #输出16个通道(每个卷积核一个)，10*10的数据。计算过程为(14-5)/1 + 1= 10
        #输出(16,10,10)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120) #全连接层，输入(16,5,5)的数据，输出120个节点
        self.fc2 = nn.Linear(120, 84) #全连接层，120个连接84个
        self.fc3 = nn.Linear(84, 10) #全连接层，84个连接10个
        #

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #第一次卷积、relu激活、池化。输出(6,14,14)的数据
        x = self.pool(F.relu(self.conv2(x))) #第二次卷积、relu激活、池化。输出(16,5,5)的数据
        x = x.view(-1, x.size()[1:].numel()) #将数据展开成一维
        x = F.relu(self.fc1(x)) #第一层全连接、relu激活
        x = F.relu(self.fc2(x)) #第二层全连接、relu激活
        x = self.fc3(x) #第三层全连接，得到评分
        return x

从本地文件夹读取数据集，但未对数据进行预处理(均值减法、归一化等)，效果不佳，通过后面的对比可以发现预处理的重要性。

In [3]:
#解码数据集 函数为CIFAR-10提供的python3版本
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#创建训练集和测试集
def CreatData(path):
    #依次加载CIFAR-10的5个batch_data,并将其合并为traindata和traindlabels
    x=[]
    y=[]
    for i in range(1,6):
        batch_path=path + 'data_batch_%d'%(i) #每个batch的地址
        batch_dict=unpickle(batch_path) #解码每个batch
        train_batch=batch_dict[b'data'].astype('float') #将每个batch的data部分以float形式存储于train_batch变量
        train_labels=np.array(batch_dict[b'labels']) #将每个batch的label部分以np.array的形式存储于train_labels变量
        x.append(train_batch)
        y.append(train_labels)
    #将5个训练样本batch(10000,3072)合并为(50000,3072)，标签合并为(50000,1)
    #np.concatenate默认axis=0:按行合并，axis=1则为:按列合并
    traindata=torch.from_numpy(np.concatenate(x)).view(-1,3,32,32)
    trainlabels=torch.from_numpy(np.concatenate(y)).long().view(-1)
    
    #加载测试集
    testpath=path + 'test_batch' #test_batch的地址
    test_dict=unpickle(testpath) #解码test_batch
    testdata=torch.from_numpy(test_dict[b'data'].astype('float')).view(-1,3,32,32) #将test_dict的data部分以float形式存储于testdata变量
    testlabels=torch.from_numpy(np.array(test_dict[b'labels'])).long().view(-1) #将test_dict的labels部分以np.array形式存储于testlabels变量
    
    #将训练集数据、训练集标签、测试集数据、测试集标签返回
    return traindata,trainlabels,testdata,testlabels

In [4]:
#调用自己定义的函数读取本地训练集和测试集的数据和标签
traindata,trainlabels,testdata,testlabels = CreatData("E:/dataset/cifar-10-batches-py/")

In [5]:
net = Net() #实例化网络
criterion = nn.CrossEntropyLoss() #交叉熵损失
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) #构造优化器

for epoch in range(2):  # 多批次循环
    running_loss = 0.0
    for i in range(traindata.shape[0]):
        # 获取输入和标签
        inputs = traindata[i].unsqueeze(0).float()
        labels = trainlabels[i].view(-1)
        # 梯度置0
        optimizer.zero_grad()

        outputs = net(inputs)#正向传播
        loss = criterion(outputs, labels)#计算损失
        loss.backward()#反向传播
        optimizer.step()#更新网络参数

        # 打印损失信息
        running_loss += loss.item()
        if i % 2000 == 1999:    # 每2000批次打印一次
            print('[',epoch + 1,',',i + 1,']','loss:',running_loss / 2000)
            running_loss = 0.0
print('Finished Training')

[ 1 , 2000 ] loss: 2.316291594982147
[ 1 , 4000 ] loss: 2.3053256458044054
[ 1 , 6000 ] loss: 2.3068840577602385
[ 1 , 8000 ] loss: 2.3047428538799286
[ 1 , 10000 ] loss: 2.305438923239708
[ 1 , 12000 ] loss: 2.3055933393239973
[ 1 , 14000 ] loss: 2.3055312781333925
[ 1 , 16000 ] loss: 2.30385791349411
[ 1 , 18000 ] loss: 2.3064646641016004
[ 1 , 20000 ] loss: 2.3064027812480927
[ 1 , 22000 ] loss: 2.3054550058841707
[ 1 , 24000 ] loss: 2.305367486000061
[ 1 , 26000 ] loss: 2.3044186067581176
[ 1 , 28000 ] loss: 2.303218544483185
[ 1 , 30000 ] loss: 2.3044489706754683
[ 1 , 32000 ] loss: 2.3029126390218733
[ 1 , 34000 ] loss: 2.3044829057455063
[ 1 , 36000 ] loss: 2.3064204243421553
[ 1 , 38000 ] loss: 2.305007672548294
[ 1 , 40000 ] loss: 2.3056939866542816
[ 1 , 42000 ] loss: 2.304836077094078
[ 1 , 44000 ] loss: 2.306043698310852
[ 1 , 46000 ] loss: 2.3044300192594527
[ 1 , 48000 ] loss: 2.3045212196111677
[ 1 , 50000 ] loss: 2.303801007926464
[ 2 , 2000 ] loss: 2.29908312189579
[ 2

In [6]:
#测试效果
correct = 0
total = 0
with torch.no_grad():#不更新梯度
    for i in range(testdata.shape[0]):
        # 获取输入
        inputs = testdata[i].unsqueeze(0).float()
        labels = testlabels[i].view(-1)
        
        output = net(inputs)#正向传播
        _, predicted = torch.max(output, 1)#预测标签
        total += labels.size(0)#测试图像数目
        correct += (predicted == labels).sum().item()#预测正确数目
print('Accuracy of the network on the 10000 test images:',100 * correct / total,'%')
#print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 10.03 %


使用pytorch的dataset和dataloader，并对数据进行一定的预处理

In [7]:
#读取训练集和测试机数据
transform = transforms.Compose([transforms.ToTensor() , transforms.Normalize((0.5, 0.5, 0.5) , (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [8]:
net = Net() #实例化网络
criterion = nn.CrossEntropyLoss() #交叉熵损失
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) #构造优化器

In [9]:
for epoch in range(2):  # 多批次循环

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 获取输入
        inputs, labels = data

        # 梯度置0
        optimizer.zero_grad()

        outputs = net(inputs)#正向传播
        loss = criterion(outputs, labels)#计算损失
        loss.backward()#反向传播
        optimizer.step()#更新网络参数

        # 打印损失信息
        running_loss += loss.item()
        if i % 2000 == 1999:    # 每2000批次打印一次
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.210
[1,  4000] loss: 1.886
[1,  6000] loss: 1.694
[1,  8000] loss: 1.592
[1, 10000] loss: 1.529
[1, 12000] loss: 1.487
[2,  2000] loss: 1.404
[2,  4000] loss: 1.359
[2,  6000] loss: 1.342
[2,  8000] loss: 1.328
[2, 10000] loss: 1.304
[2, 12000] loss: 1.255
Finished Training


In [10]:
#训练集准确率
correct = 0
total = 0
with torch.no_grad():#不计算梯度
    for data in trainloader:
        images, labels = data#载入训练数据
        outputs = net(images)#正向传播
        _, predicted = torch.max(outputs.data, 1)#预测标签
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 58 %


In [11]:
#测试集准确率
correct = 0
total = 0
with torch.no_grad():#不计算梯度
    for data in testloader:
        images, labels = data#载入训练数据
        outputs = net(images)#正向传播
        _, predicted = torch.max(outputs.data, 1)#预测标签
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 56 %
